# Tweaked ResNet50  for 48x48x1 inputs

Made few chages in the first convolution and pooling layer of ResNet50 such that initial infomation loss is avoided for small (48x48) size input image

References:  
&nbsp;&nbsp;\[1\] [Resnet with Keras](https://github.com/priya-dwivedi/Deep-Learning/blob/master/resnet_keras/Residual_Networks_yourself.ipynb)  
&nbsp;&nbsp;\[2\] [Kaggle Facial Expression DataSet](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge) is of grayscale 48x48 size images

In [0]:
import numpy as np
import keras

Using TensorFlow backend.


### Define Residual(=Identity) Bock

In [0]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = keras.layers.Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', 
                            kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = keras.layers.Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = keras.layers.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', 
                            kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = keras.layers.Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', 
                            kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)
    
    
    return X

### Define Down-sampling(=Convolution) Bock

In [0]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = keras.layers.Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', 
                            kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = keras.layers.Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = keras.layers.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', 
                            kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = keras.layers.Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', 
                            kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = keras.initializers.glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)
    
    return X

### Define ResNet50

In [0]:
def get_resnet50_48x48(input_shape=(48, 48, 1), classes=7):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = keras.layers.Input(input_shape)

    # Zero-Padding
    X = keras.layers.ZeroPadding2D((2, 2))(X_input)

    # Stage 1
    X = keras.layers.Conv2D(64, (5, 5), strides=(1, 1), name='conv1', 
		                        kernel_initializer=keras.initializers.glorot_uniform(seed=0))(X)
    X = keras.layers.BatchNormalization(axis=3, name='bn_conv1')(X)
    X = keras.layers.Activation('relu')(X)	
    # X = keras.layers.ZeroPadding2D((1, 1))(X)


    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    # X = keras.layers.AveragePooling2D((2,2), name="avg_pool")(X)
    # X = keras.layers.GlobalAveragePooling2D(name="avg_pool")(X)

    ### END CODE HERE ###

    # Create model
    model = keras.models.Model(inputs = X_input, outputs = X, name='resnet50_48x48')

    return model

In [0]:
resnet50_48x48 = get_resnet50_48x48()
resnet50_48x48.summary()

W0814 20:48:54.113225 139818215053184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 20:48:54.133656 139818215053184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 20:48:54.139347 139818215053184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0814 20:48:54.162993 139818215053184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0814 20:48:54.163847 1398182150

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 52, 52, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   1664        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
# model = keras.applications.resnet50.ResNet50(weights='imagenet')
# model.summary()

### Add subnet for classification

In [0]:
model = keras.models.Sequential()
model.add(resnet50_48x48)
model.add(keras.layers.GlobalAveragePooling2D(name='global_ave_pool'))
model.add(keras.layers.Dense(units=2048,name='FC1'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(units=128,name='FC2'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(7, activation='softmax'))
model.summary()

W0814 20:49:03.406567 139818215053184 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50_48x48 (Model)       (None, 6, 6, 2048)        23579904  
_________________________________________________________________
global_ave_pool (GlobalAvera (None, 2048)              0         
_________________________________________________________________
FC1 (Dense)                  (None, 2048)              4196352   
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
activation_50 (Activation)   (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
FC2 (Dense)                  (None, 128)               262272    
__________

In [0]:
# model.add(keras.layers.Conv2D(filters=256),kernal_size=(3,3),name='classification_conv1')
# model.add(keras.layers.Activation('relu'))
# model.add(keras.layers.Conv2D(filters=256),kernal_size=(3,3),name='classification_conv2')
# model.add(keras.layers.Activation('relu'))
# model.add(keras.layers.Conv2D(filters=256),kernal_size=(3,3),name='classification_conv3')
# model.add(keras.layers.Activation('relu'))
# model.add(keras.layers.Conv2D(filters=256),kernal_size=(3,3),name='classification_conv4')
# model.add(keras.layers.Activation('relu'))
# model.add(keras.layers.Conv2D(filters=7*9),kernal_size=(3,3),name='classification_conv5')

### Load Dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
data_path = '/content/drive/My Drive/Colab Notebooks/data/'

Mounted at /content/drive/


In [0]:
import pickle
with open(data_path+'FER2013.train_data.pickle','rb') as f:
  train_data = pickle.load(f)
with open(data_path+'FER2013.public_test_data.pickle','rb') as f:
  public_test_data = pickle.load(f)
with open(data_path+'FER2013.private_test_data.pickle','rb') as f:
  private_test_data = pickle.load(f)

### Train

In [0]:
check_point = keras.callbacks.ModelCheckpoint(data_path+'ResNet50_48x48.epoch={epoch:02d}.acc={acc:.2f}.val_acc={val_acc:.2f}.hdf5', monitor='val_loss', verbose=0, 
                                              save_best_only=True, save_weights_only=False, mode='auto', period=1)
stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(train_data['x'], train_data['y'],
                    validation_data=(public_test_data['x'], public_test_data['y']),
                    epochs = 100, batch_size = 128, callbacks=[check_point,stopping])

with open(data_path+'ResNet50_48x48.hist.pickle','wb') as f:
  pickle.dump(history,f)

W0814 20:49:12.520902 139818215053184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0814 20:49:12.701581 139818215053184 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 28709 samples, validate on 3589 samples
Epoch 1/100
28709/28709 [==============================] - 257s 9ms/step - loss: 1.8364 - acc: 0.2490 - val_loss: 1.7471 - val_acc: 0.2859
Epoch 2/100
28709/28709 [==============================] - 242s 8ms/step - loss: 1.5652 - acc: 0.3926 - val_loss: 6.2049 - val_acc: 0.2820
Epoch 3/100
28709/28709 [==============================] - 243s 8ms/step - loss: 1.3760 - acc: 0.4738 - val_loss: 1.7690 - val_acc: 0.2990
Epoch 4/100
28709/28709 [==============================] - 243s 8ms/step - loss: 1.2518 - acc: 0.5200 - val_loss: 2.2649 - val_acc: 0.3018
Epoch 5/100
28709/28709 [==============================] - 243s 8ms/step - loss: 1.1663 - acc: 0.5536 - val_loss: 1.4729 - val_acc: 0.4915
Epoch 6/100
28709/28709 [==============================] - 242s 8ms/step - loss: 1.0927 - acc: 0.5877 - val_loss: 2.3603 - val_acc: 0.2923
Epoch 7/100
28709/28709 [==============================] - 243s 8ms/step - loss: 1.0154 - acc: 0.6197 - val_loss: 1.2